In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import time

In [2]:
# Initialise the random state
num = random.randint(1, 500)
print(f"The generated random seed is {num}") #347

The generated random seed is 54


In [3]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data/EMG_data_collection/combined_stats_nihara.csv"
data = pd.read_csv(path)
data.head()

,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1,max_Ax_Hand0,max_Ay_Hand0,max_Az_Hand0,max_Vx_Hand0,max_Vy_Hand0,max_Vz_Hand0,max_Jx_Hand0,max_Jy_Hand0,max_Jz_Hand0,mean_Ax_Hand0,mean_Ay_Hand0,mean_Az_Hand0,mean_Vx_Hand0,mean_Vy_Hand0,mean_Vz_Hand0,mean_Jx_Hand0,mean_Jy_Hand0,mean_Jz_Hand0,std_Ax_Hand0,std_Ay_Hand0,std_Az_Hand0,std_Vx_Hand0,std_Vy_Hand0,std_Vz_Hand0,std_Jx_Hand0,std_Jy_Hand0,std_Jz_Hand0,max_Ax_Hand1,max_Ay_Hand1,max_Az_Hand1,max_Vx_Hand1,max_Vy_Hand1,max_Vz_Hand1,max_Jx_Hand1,max_Jy_Hand1,max_Jz_Hand1,mean_Ax_Hand1,mean_Ay_Hand1,mean_Az_Hand1,mean_Vx_Hand1,mean_Vy_Hand1,mean_Vz_Hand1,mean_Jx_Hand1,mean_Jy_Hand1,mean_Jz_Hand1,std_Ax_Hand1,std_Ay_Hand1,std_Az_Hand1,std_Vx_Hand1,std_Vy_Hand1,std_Vz_Hand1,std_Jx_Hand1,std_Jy_Hand1,std_Jz_Hand1,max_w_Hand0,max_x_Hand0,max_y_Hand0,max_z_Hand0,max_AVx_Hand0,max_AVy_Hand0,max_AVz_Hand0,max_AAx_Hand0,max_AAy_Hand0,max_AAz_Hand0,max_AJx_Hand0,max_AJy_Hand0,max_AJz_Hand0,mean_w_Hand0,mean_x_Hand0,mean_y_Hand0,mean_z_Hand0,mean_AVx_Hand0,mean_AVy_Hand0,mean_AVz_Hand0,mean_AAx_Hand0,mean_AAy_Hand0,mean_AAz_Hand0,mean_AJx_Hand0,mean_AJy_Hand0,mean_AJz_Hand0,std_w_Hand0,std_x_Hand0,std_y_Hand0,std_z_Hand0,std_AVx_Hand0,std_AVy_Hand0,std_AVz_Hand0,std_AAx_Hand0,std_AAy_Hand0,std_AAz_Hand0,std_AJx_Hand0,std_AJy_Hand0,std_AJz_Hand0,max_w_Hand1,max_x_Hand1,max_y_Hand1,max_z_Hand1,max_AVx_Hand1,max_AVy_Hand1,max_AVz_Hand1,max_AAx_Hand1,max_AAy_Hand1,max_AAz_Hand1,max_AJx_Hand1,max_AJy_Hand1,max_AJz_Hand1,mean_w_Hand1,mean_x_Hand1,mean_y_Hand1,mean_z_Hand1,mean_AVx_Hand1,mean_AVy_Hand1,mean_AVz_Hand1,mean_AAx_Hand1,mean_AAy_Hand1,mean_AAz_Hand1,mean_AJx_Hand1,mean_AJy_Hand1,mean_AJz_Hand1,std_w_Hand1,std_x_Hand1,std_y_Hand1,std_z_Hand1,std_AVx_Hand1,std_AVy_Hand1,std_AVz_Hand1,std_AAx_Hand1,std_AAy_Hand1,std_AAz_Hand1,std_AJx_Hand1,std_AJy_Hand1,std_AJz_Hand1
0,19,3,16,softness,119,6,high,3,1,2022-02-13 11:24:11.302,2022-02-13 11:24:16.299,0.242581,0.158296,0.130757,0.060673,0.115441,0.219388,0.151590,0.152921,0.013684,0.010396,0.007389,0.006122,0.008396,0.018439,0.015110,0.017840,0.019036,0.014008,0.007698,0.005684,0.009616,0.022691,0.015458,0.016931,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.009221,0.006795,0.013644,0.010730,0.007936,0.028964,0.020745,0.014779,0.013452,0.006873,0.018450,0.016978,0.013093,0.031076,0.024360,0.017118,1.225601,0.131363,0.834671,0.038557,1.048052,1.312881,0.668174,1.758989,2.569901,0.806518,-0.054221,0.591324,-0.003221,-0.223929,-0.153043,0.003040,-0.012368,-0.015513,0.082628,0.034411,0.056503,0.018108,0.479674,0.552304,0.050005,0.197537,0.348330,0.857727,0.186783,0.988373,0.039914,3.557402,0.448096,2.758789,1.902613,0.411489,0.634328,0.033188,0.775963,-0.002446,0.256799,-0.209355,0.011060,0.025355,-0.061193,0.104347,0.061546,0.073134,0.007845,0.573429,0.553135,0.178206,0.261626,0.370657,0.272057,0.459350,-0.057506,0.912503,1.028816,1.656573,0.482456,39.494275,35.226491,125.211266,3638.365630,4263.032202,4319.828070,0.247198,0.424915,-0.156655,0.855569,0.011910,0.009762,-0.094809,0.053775,-0.004176,0.472270,-17.716852,-20.931510,-14.542370,0.029243,0.017665,0.020427,0.014979,0.262721,0.336230,0.439350,8.068401,9.776817,13.178315,723.8

In [4]:
# Obtain only the physical properties
physical_data = data[data.property_name != 'enjoyment']
physical_data.reset_index(inplace=True, drop = True)
physical_data.head()

,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1,max_Ax_Hand0,max_Ay_Hand0,max_Az_Hand0,max_Vx_Hand0,max_Vy_Hand0,max_Vz_Hand0,max_Jx_Hand0,max_Jy_Hand0,max_Jz_Hand0,mean_Ax_Hand0,mean_Ay_Hand0,mean_Az_Hand0,mean_Vx_Hand0,mean_Vy_Hand0,mean_Vz_Hand0,mean_Jx_Hand0,mean_Jy_Hand0,mean_Jz_Hand0,std_Ax_Hand0,std_Ay_Hand0,std_Az_Hand0,std_Vx_Hand0,std_Vy_Hand0,std_Vz_Hand0,std_Jx_Hand0,std_Jy_Hand0,std_Jz_Hand0,max_Ax_Hand1,max_Ay_Hand1,max_Az_Hand1,max_Vx_Hand1,max_Vy_Hand1,max_Vz_Hand1,max_Jx_Hand1,max_Jy_Hand1,max_Jz_Hand1,mean_Ax_Hand1,mean_Ay_Hand1,mean_Az_Hand1,mean_Vx_Hand1,mean_Vy_Hand1,mean_Vz_Hand1,mean_Jx_Hand1,mean_Jy_Hand1,mean_Jz_Hand1,std_Ax_Hand1,std_Ay_Hand1,std_Az_Hand1,std_Vx_Hand1,std_Vy_Hand1,std_Vz_Hand1,std_Jx_Hand1,std_Jy_Hand1,std_Jz_Hand1,max_w_Hand0,max_x_Hand0,max_y_Hand0,max_z_Hand0,max_AVx_Hand0,max_AVy_Hand0,max_AVz_Hand0,max_AAx_Hand0,max_AAy_Hand0,max_AAz_Hand0,max_AJx_Hand0,max_AJy_Hand0,max_AJz_Hand0,mean_w_Hand0,mean_x_Hand0,mean_y_Hand0,mean_z_Hand0,mean_AVx_Hand0,mean_AVy_Hand0,mean_AVz_Hand0,mean_AAx_Hand0,mean_AAy_Hand0,mean_AAz_Hand0,mean_AJx_Hand0,mean_AJy_Hand0,mean_AJz_Hand0,std_w_Hand0,std_x_Hand0,std_y_Hand0,std_z_Hand0,std_AVx_Hand0,std_AVy_Hand0,std_AVz_Hand0,std_AAx_Hand0,std_AAy_Hand0,std_AAz_Hand0,std_AJx_Hand0,std_AJy_Hand0,std_AJz_Hand0,max_w_Hand1,max_x_Hand1,max_y_Hand1,max_z_Hand1,max_AVx_Hand1,max_AVy_Hand1,max_AVz_Hand1,max_AAx_Hand1,max_AAy_Hand1,max_AAz_Hand1,max_AJx_Hand1,max_AJy_Hand1,max_AJz_Hand1,mean_w_Hand1,mean_x_Hand1,mean_y_Hand1,mean_z_Hand1,mean_AVx_Hand1,mean_AVy_Hand1,mean_AVz_Hand1,mean_AAx_Hand1,mean_AAy_Hand1,mean_AAz_Hand1,mean_AJx_Hand1,mean_AJy_Hand1,mean_AJz_Hand1,std_w_Hand1,std_x_Hand1,std_y_Hand1,std_z_Hand1,std_AVx_Hand1,std_AVy_Hand1,std_AVz_Hand1,std_AAx_Hand1,std_AAy_Hand1,std_AAz_Hand1,std_AJx_Hand1,std_AJy_Hand1,std_AJz_Hand1
0,19,3,16,softness,119,6,high,3,1,2022-02-13 11:24:11.302,2022-02-13 11:24:16.299,0.242581,0.158296,0.130757,0.060673,0.115441,0.219388,0.151590,0.152921,0.013684,0.010396,0.007389,0.006122,0.008396,0.018439,0.015110,0.017840,0.019036,0.014008,0.007698,0.005684,0.009616,0.022691,0.015458,0.016931,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.009221,0.006795,0.013644,0.010730,0.007936,0.028964,0.020745,0.014779,0.013452,0.006873,0.018450,0.016978,0.013093,0.031076,0.024360,0.017118,1.225601,0.131363,0.834671,0.038557,1.048052,1.312881,0.668174,1.758989,2.569901,0.806518,-0.054221,0.591324,-0.003221,-0.223929,-0.153043,0.003040,-0.012368,-0.015513,0.082628,0.034411,0.056503,0.018108,0.479674,0.552304,0.050005,0.197537,0.348330,0.857727,0.186783,0.988373,0.039914,3.557402,0.448096,2.758789,1.902613,0.411489,0.634328,0.033188,0.775963,-0.002446,0.256799,-0.209355,0.011060,0.025355,-0.061193,0.104347,0.061546,0.073134,0.007845,0.573429,0.553135,0.178206,0.261626,0.370657,0.272057,0.459350,-0.057506,0.912503,1.028816,1.656573,0.482456,39.494275,35.226491,125.211266,3638.365630,4263.032202,4319.828070,0.247198,0.424915,-0.156655,0.855569,0.011910,0.009762,-0.094809,0.053775,-0.004176,0.472270,-17.716852,-20.931510,-14.542370,0.029243,0.017665,0.020427,0.014979,0.262721,0.336230,0.439350,8.068401,9.776817,13.178315,723.8

In [5]:
physical_data.insert(3, 'new_property_id', 20)
physical_data['new_property_id'] = physical_data['property_id'] - 12

physical_data.insert(9, 'new_rating_level_num', 10)
physical_data['new_rating_level_num'] = physical_data['rating_level_num'] - 1
physical_data.head()

,participant_id,clothes_id,property_id,new_property_id,property_name,interaction_id,rating,rating_level,rating_level_num,new_rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1,max_Ax_Hand0,max_Ay_Hand0,max_Az_Hand0,max_Vx_Hand0,max_Vy_Hand0,max_Vz_Hand0,max_Jx_Hand0,max_Jy_Hand0,max_Jz_Hand0,mean_Ax_Hand0,mean_Ay_Hand0,mean_Az_Hand0,mean_Vx_Hand0,mean_Vy_Hand0,mean_Vz_Hand0,mean_Jx_Hand0,mean_Jy_Hand0,mean_Jz_Hand0,std_Ax_Hand0,std_Ay_Hand0,std_Az_Hand0,std_Vx_Hand0,std_Vy_Hand0,std_Vz_Hand0,std_Jx_Hand0,std_Jy_Hand0,std_Jz_Hand0,max_Ax_Hand1,max_Ay_Hand1,max_Az_Hand1,max_Vx_Hand1,max_Vy_Hand1,max_Vz_Hand1,max_Jx_Hand1,max_Jy_Hand1,max_Jz_Hand1,mean_Ax_Hand1,mean_Ay_Hand1,mean_Az_Hand1,mean_Vx_Hand1,mean_Vy_Hand1,mean_Vz_Hand1,mean_Jx_Hand1,mean_Jy_Hand1,mean_Jz_Hand1,std_Ax_Hand1,std_Ay_Hand1,std_Az_Hand1,std_Vx_Hand1,std_Vy_Hand1,std_Vz_Hand1,std_Jx_Hand1,std_Jy_Hand1,std_Jz_Hand1,max_w_Hand0,max_x_Hand0,max_y_Hand0,max_z_Hand0,max_AVx_Hand0,max_AVy_Hand0,max_AVz_Hand0,max_AAx_Hand0,max_AAy_Hand0,max_AAz_Hand0,max_AJx_Hand0,max_AJy_Hand0,max_AJz_Hand0,mean_w_Hand0,mean_x_Hand0,mean_y_Hand0,mean_z_Hand0,mean_AVx_Hand0,mean_AVy_Hand0,mean_AVz_Hand0,mean_AAx_Hand0,mean_AAy_Hand0,mean_AAz_Hand0,mean_AJx_Hand0,mean_AJy_Hand0,mean_AJz_Hand0,std_w_Hand0,std_x_Hand0,std_y_Hand0,std_z_Hand0,std_AVx_Hand0,std_AVy_Hand0,std_AVz_Hand0,std_AAx_Hand0,std_AAy_Hand0,std_AAz_Hand0,std_AJx_Hand0,std_AJy_Hand0,std_AJz_Hand0,max_w_Hand1,max_x_Hand1,max_y_Hand1,max_z_Hand1,max_AVx_Hand1,max_AVy_Hand1,max_AVz_Hand1,max_AAx_Hand1,max_AAy_Hand1,max_AAz_Hand1,max_AJx_Hand1,max_AJy_Hand1,max_AJz_Hand1,mean_w_Hand1,mean_x_Hand1,mean_y_Hand1,mean_z_Hand1,mean_AVx_Hand1,mean_AVy_Hand1,mean_AVz_Hand1,mean_AAx_Hand1,mean_AAy_Hand1,mean_AAz_Hand1,mean_AJx_Hand1,mean_AJy_Hand1,mean_AJz_Hand1,std_w_Hand1,std_x_Hand1,std_y_Hand1,std_z_Hand1,std_AVx_Hand1,std_AVy_Hand1,std_AVz_Hand1,std_AAx_Hand1,std_AAy_Hand1,std_AAz_Hand1,std_AJx_Hand1,std_AJy_Hand1,std_AJz_Hand1
0,19,3,16,4,softness,119,6,high,3,2,1,2022-02-13 11:24:11.302,2022-02-13 11:24:16.299,0.242581,0.158296,0.130757,0.060673,0.115441,0.219388,0.151590,0.152921,0.013684,0.010396,0.007389,0.006122,0.008396,0.018439,0.015110,0.017840,0.019036,0.014008,0.007698,0.005684,0.009616,0.022691,0.015458,0.016931,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.009221,0.006795,0.013644,0.010730,0.007936,0.028964,0.020745,0.014779,0.013452,0.006873,0.018450,0.016978,0.013093,0.031076,0.024360,0.017118,1.225601,0.131363,0.834671,0.038557,1.048052,1.312881,0.668174,1.758989,2.569901,0.806518,-0.054221,0.591324,-0.003221,-0.223929,-0.153043,0.003040,-0.012368,-0.015513,0.082628,0.034411,0.056503,0.018108,0.479674,0.552304,0.050005,0.197537,0.348330,0.857727,0.186783,0.988373,0.039914,3.557402,0.448096,2.758789,1.902613,0.411489,0.634328,0.033188,0.775963,-0.002446,0.256799,-0.209355,0.011060,0.025355,-0.061193,0.104347,0.061546,0.073134,0.007845,0.573429,0.553135,0.178206,0.261626,0.370657,0.272057,0.459350,-0.057506,0.912503,1.028816,1.656573,0.482456,39.494275,35.226491,125.211266,3638.365630,4263.032202,4319.828070,0.247198,0.424915,-0.156655,0.855569,0.011910,0.009762,-0.094809,0.053775,-0.004176,0.472270,-17.716852,-20.931510,-14.542370,0.029243,0.017665,0.020427,0.014979,0.262721,0.336230,0

In [6]:
# Obtain only the EMG data
ending_index = 61
emg_data = physical_data.iloc[:, :ending_index]
emg_data

,participant_id,clothes_id,property_id,new_property_id,property_name,interaction_id,rating,rating_level,rating_level_num,new_rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1
0,19,3,16,4,softness,119,6,high,3,2,1,2022-02-13 11:24:11.302,2022-02-13 11:24:16.299,0.242581,0.158296,0.130757,0.060673,0.115441,0.219388,0.151590,0.152921,0.013684,0.010396,0.007389,0.006122,0.008396,0.018439,0.015110,0.017840,0.019036,0.014008,0.007698,0.005684,0.009616,0.022691,0.015458,0.016931,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.009221,0.006795,0.013644,0.010730,0.007936,0.028964,0.020745,0.014779,0.013452,0.006873,0.018450,0.016978,0.013093,0.031076,0.024360,0.017118
1,19,3,16,4,softness,119,6,high,3,2,2,2022-02-13 11:24:16.341,2022-02-13 11:24:21.262,0.293434,0.158296,0.134725,0.123295,0.260187,0.358265,0.190598,0.290818,0.015520,0.006953,0.007818,0.007263,0.010123,0.021377,0.015354,0.020431,0.023535,0.010096,0.010208,0.009635,0.015149,0.032378,0.017204,0.022866,0.110325,0.043661,0.107576,0.178948,0.225171,0.226867,0.163749,0.161080,0.006464,0.005104,0.011004,0.008545,0.008792,0.026416,0.016349,0.013655,0.007602,0.004338,0.009174,0.010905,0.015095,0.028296,0.017766,0.013311
2,19,3,16,4,softness,119,6,high,3,2,3,2022-02-13 11:24:21.311,2022-02-13 11:24:27.250,0.159430,0.060233,0.101259,0.096457,0.219979,0.235000,0.151590,0.172621,0.008906,0.005245,0.006299,0.006227,0.010214,0.020416,0.015191,0.017304,0.012227,0.004520,0.005452,0.006955,0.016094,0.023721,0.016089,0.017499,0.132567,0.064446,0.144146,0.304368,0.234884,0.357279,0.228678,0.106441,0.006702,0.005303,0.011059,0.008533,0.007853,0.026675,0.016720,0.013531,0.007471,0.004612,0.009831,0.012904,0.015809,0.029737,0.017870,0.012967
3,19,3,15,3,flexibility,120,5,medium,2,1,1,2022-02-13 11:27:08.930,2022-02-13 11:27:13.930,0.305616,0.452484,0.140589,0.114349,0.155648,0.422347,0.229605,0.231719,0.026890,0.014963,0.010571,0.010014,0.017151,0.033179,0.025193,0.029496,0.032213,0.023242,0.012242,0.010140,0.018083,0.033755,0.025715,0.027528,0.266018,0.392933,0.325735,0.505040,0.267168,0.536120,0.309869,0.243039,0.016434,0.016481,0.026583,0.017290,0.018351,0.053903,0.038799,0.024986,0.019200,0.024426,0.033167,0.027475,0.022279,0.051795,0.036870,0.026440
4,19,3,15,3,flexibility,120,5,medium,2,1,2,2022-02-13 11:27:13.949,2022-02-13 11:27:18.905,0.815143,0.256359,0.301881,0.127192,0.155648,0.437959,0.288116,0.428714,0.044400,0.015118,0.017596,0.012659,0.015900,0.036974,0.025262,0.035987,0.075118,0.020009,0.027983,0.015224,0.018707,0.042568,0.031915,0.039454,0.288260,0.143453,0.194490,0.209854,0.525445,0.460363,0.350465,0.243039,0.024463,0.013069,0.022481,0.015321,0.015366,0.049206,0.038742,0.025788,0.030630,0.013435,0.024008,0.018971,0.028355,0.053952,0.046198,0.026669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,25,6,15,3,flexibility,355,4,medium,2,1,2,2022-02-15 12:02:14.400,2022-02-15 12:02:19.365,0.337148,0.420763,0.260693,1.000000,0.992090,0.191503,0.572555,0.490479,0.012292,0.016161,0.013291,0.044361,0.093415,0.020323,0.029711,0.031834,0.020175,0.028164,0.021847,0.1

In [7]:
print(emg_data.property_name.unique())
print(emg_data.new_property_id.unique())

['softness' 'flexibility' 'smoothness' 'thickness' 'warmth']
[4 3 0 1 2]


In [8]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1) #3, 6, 5
        self.pool = nn.MaxPool2d(2, 2) # Kernel size of 2 and a stride of 2
        self.conv2 = nn.Conv1d(in_channels=12, out_channels=48, kernel_size=3, stride=1, padding=1) #3, 6, 5
        self.rnn = nn.LSTM(12, 18, 1)
        self.h0 = torch.randn(1, 24, 18)
        self.c0 = torch.randn(1, 24, 18)
        self.fc1 = nn.Linear(24 * 18, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)
        
   
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x, (hn, cn) = self.rnn(x, (self.h0, self.c0))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        

In [9]:
# Summarise and print the network architechture
m = CNN_LSTM()
print(f"Summary of the network architechture: {m}")

Summary of the network architechture: CNN_LSTM(
  (conv1): Conv1d(3, 24, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(12, 48, kernel_size=(3,), stride=(1,), padding=(1,))
  (rnn): LSTM(12, 18)
  (fc1): Linear(in_features=432, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)


In [10]:
def data_preparation(emg_data, starting_index = 13, predicting_feature='new_property_id', random_state=num):
    
    X_arr = emg_data.iloc[:,starting_index:].values
    print(f'X_arr before reshaping {X_arr.shape}')
    X_arr = X_arr.reshape((int(X_arr.shape[0]/3), 3, int(X_arr.shape[1])),order='F')
    print(f'X_arr after reshaping {X_arr.shape}')
    
    #y_arr = emg_data.new_property_id.values
    y_arr = emg_data[predicting_feature].values
    y_arr = y_arr[0::3]
    print(f'y_arr before reshaping {y_arr.shape}')
    y_arr = np.reshape(y_arr, (y_arr.shape[0],1))
    print(f'y_arr after reshaping {y_arr.shape}')
    
    # Create training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size = 0.1, shuffle=True, random_state = num) 

    # transform arrays to torch tensors
    tensor_X_train = torch.Tensor(X_train) 
    tensor_X_test = torch.Tensor(X_test) 
    tensor_y_train = torch.Tensor(y_train)
    tensor_y_test = torch.Tensor(y_test)
    
    return tensor_X_train, tensor_X_test, tensor_y_train, tensor_y_test





In [11]:
def training(X_train, y_train):
    
    batch_size = 54
    train_dataset = TensorDataset(X_train, y_train) 
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    ## Model
    model = CNN_LSTM()


    ## Loss and Optimiser
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


    ## Train
    print('Starting training.')
    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            labels = labels.reshape([batch_size])
            labels = labels.type(torch.LongTensor)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Update statistics
            running_loss += loss.item()

    print('Training done.')

    # save trained model    
    torch.save(model.state_dict(), 'saved_model.pt')
    print('Model saved.')

In [12]:
#if __name__ == '__main__':
def testing(X_test, y_test, predicting_feature='new_property_id'):

    batch_size = 18
    test_dataset = TensorDataset(X_test, y_test) 
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    dataiter = iter(test_dataloader)
    #if predicting_feature=='new_property_id':
       # classes = ('smoothness', 'thickness', 'warmth', 'flexibility', 'softness')
    #elif predicting_feature == 'new_rating_level_num':
        #classes = (1,2,3,4,5,6,7)


    ## load the trained model
    class_model = CNN_LSTM()
    class_model.load_state_dict(torch.load('saved_model.pt'))


    ## inference
    images, labels = dataiter.next()
    labels = labels.reshape([batch_size])
    labels = labels.type(torch.LongTensor)
    #print('Ground-truth: ', ' '.join('%5s' % classes[labels_3[j]] for j in range(batch_size)))

    outputs = class_model(images)
    _, predicted = torch.max(outputs, 1)
    #print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(batch_size)))
    
    labels_np = labels.numpy()
    predicted_np = predicted.numpy()
    
    if predicting_feature=='new_property_id':
        conf_mat = confusion_matrix(labels_np, predicted_np, labels=[0, 1, 2, 3, 4])
        macro_f1_score = f1_score(labels_np, predicted_np, average='macro') 
    elif predicting_feature=='new_rating_level_num':
        conf_mat = confusion_matrix(labels_np, predicted_np, labels=[0, 1, 2]) 
        weighted_f1_score = f1_score(labels_np, predicted_np, average='weighted') 
    micro_f1_score = f1_score(labels_np, predicted_np, average='micro')  
    acc = accuracy_score(labels_np, predicted_np)
     
    if predicting_feature == 'new_property_id':
        print(f'Using a CNN+LSTM for automatic property recognition:') 
    elif predicting_feature == 'new_rating_level_num':
        print(f'Using a CNN+LSTM for automatic rating prediction:')    
    print("(1) Confusion matrix:\n", conf_mat)
    print(f"(2) micro F1 score = {round(micro_f1_score,2)}")
    if predicting_feature == 'new_property_id':
        print(f"(3) macro F1 score = {round(macro_f1_score,2)}")
    elif predicting_feature == 'new_rating_level_num':
        print(f"(3) weighted F1 score = {round(weighted_f1_score,2)}")      
    print(f"(4) Percentage Classification accuracy = {round(acc*100,2)}%")



    




In [13]:
# Predicting properties

# Create the training and testing data
X_train, X_test, y_train, y_test = data_preparation(emg_data, starting_index = 13, predicting_feature='new_property_id', random_state=num)
print('----------------------------')
# Train the model
training(X_train, y_train)
print('----------------------------')
# Test the model
testing(X_test, y_test, predicting_feature='new_property_id')


X_arr before reshaping (540, 48)
X_arr after reshaping (180, 3, 48)
y_arr before reshaping (180,)
y_arr after reshaping (180, 1)
----------------------------
Starting training.
Training done.
Model saved.
----------------------------
Using a CNN+LSTM for automatic property recognition:
(1) Confusion matrix:
 [[0 0 0 0 4]
 [0 0 0 0 3]
 [0 0 0 0 6]
 [0 0 0 0 4]
 [0 0 0 0 1]]
(2) micro F1 score = 0.06
(3) macro F1 score = 0.02
(4) Percentage Classification accuracy = 5.56%


In [14]:
# Predicting rating
# Create the training and testing data
X_train, X_test, y_train, y_test = data_preparation(emg_data, starting_index = 13, predicting_feature='new_rating_level_num', random_state=num)
print('----------------------------')
# Train the model
training(X_train, y_train)
print('----------------------------')
# Test the model
testing(X_test, y_test, predicting_feature='new_rating_level_num')



X_arr before reshaping (540, 48)
X_arr after reshaping (180, 3, 48)
y_arr before reshaping (180,)
y_arr after reshaping (180, 1)
----------------------------
Starting training.
Training done.
Model saved.
----------------------------
Using a CNN+LSTM for automatic rating prediction:
(1) Confusion matrix:
 [[0 1 0]
 [0 9 0]
 [0 8 0]]
(2) micro F1 score = 0.5
(3) weighted F1 score = 0.33
(4) Percentage Classification accuracy = 50.0%


In [15]:
fchgvjhbkjnl

NameError: name 'fchgvjhbkjnl' is not defined

In [ ]:
drytfuygiuhi

In [ ]:
train_dataset = TensorDataset(tensor_X_train, tensor_y_train) 
train_dataloader = DataLoader(train_dataset) 

def foo():
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
            # get the inputs; data is a list of [inputs, labels]
        if i in range(2):
            b = labels




            print(inputs.shape,labels.shape, labels)
            print('-----------------------------------------------------------------------------------------------------------')
            return b

c= foo()
c.val

In [ ]:
type(c.item())

In [ ]:
     self.conv1 = nn.Conv1d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1) #3, 6, 5
        self.pool = nn.MaxPool2d(2, 2) # Kernel size of 2 and a stride of 2
        self.conv2 = nn.Conv1d(in_channels=12, out_channels=48, kernel_size=3, stride=1, padding=1) #3, 6, 5
        self.rnn = nn.LSTM(12, 18, 1)
        self.h0 = torch.randn(1, 24, 18)
        self.c0 = torch.randn(1, 24, 18)
        self.fc1 = nn.Linear(12 * 18, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)


In [ ]:




inpt = np.random.rand(1,3,48)
input_tensor = torch.Tensor(inpt)

conv1 = nn.Conv1d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1) #3, 6, 5
pool = nn.MaxPool2d(2, 2) # Kernel size of 2 and a stride of 2
conv2 = nn.Conv1d(in_channels=12, out_channels=48, kernel_size=3, stride=1, padding=1) #3, 6, 5
rnn = nn.LSTM(12, 18, 1)
h0 = torch.randn(1, 24, 18)
c0 = torch.randn(1, 24, 18)
fc1 = nn.Linear(24 * 18, 120)
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 5)


  


x = pool(F.relu(conv1(input_tensor)))
print(x.shape)
x = pool(F.relu(conv2(x)))
print(x.shape)

x, (hn, cn) = rnn(x, (h0, c0))
print(x.shape)
x = torch.flatten(x, 1) # flatten all dimensions except batch
print(x.shape)
x = F.relu(fc1(x))
print(x.shape)
x = F.relu(fc2(x))
print(x.shape)
x = fc3(x)
print(x.shape)








      #  x = self.pool(F.relu(self.conv1(x)))

In [ ]:
cghvjbknl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        print(x.shape)
        print(x)
        return x


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image    


if __name__ == '__main__':
    ## cifar-10 dataset
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    batch_size = 20
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    # example images
    dataiter = iter(trainloader)
    images, labels = dataiter.next()

    im = Image.fromarray((torch.cat(images.split(1,0),3).squeeze()/2*255+.5*255).permute(1,2,0).numpy().astype('uint8'))
    im.save("train_pt_images.jpg")
    print('train_pt_images.jpg saved.')
    print('Ground truth labels:' + ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))


    ## cnn
    net = Net()


    ## loss and optimiser
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


    ## train
    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            if i in range(2):
                print(labels)
                print(labels.shape)
                
    

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image   
if __name__ == '__main__':
    ## cifar-10 dataset
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    batch_size = 4

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    dataiter = iter(testloader)
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


    ## load the trained model
    #model = Net()
    #model.load_state_dict(torch.load('saved_model.pt'))


    ## inference
    images, labels = dataiter.next()
    print(labels)
    print(labels.shape)
    print('Ground-truth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

    #outputs = model(images)
    #_, predicted = torch.max(outputs, 1)
    #print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

    # save to images
    #im = Image.fromarray((torch.cat(images.split(1,0),3).squeeze()/2*255+.5*255).permute(1,2,0).numpy().astype('uint8'))
    #im.save("test_pt_images.jpg")
    #print('test_pt_images.jpg saved.')

In [ ]:
batch_size = 54
train_dataset = TensorDataset(tensor_X_train, tensor_y_train) 
#train_dataloader = DataLoader(train_dataset) 
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
classes = (12.0,13.0,14.0,15.0,16.0)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        print(labels.shape)
        labels_2 = labels.reshape([batch_size])
        #if i in range(2):
           # print(labels_2)
           # print(labels_2.shape)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image

#from network_pt import Net

if __name__ == '__main__':
    ## cifar-10 dataset
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    batch_size = 20
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    # example images
    dataiter = iter(trainloader)
    images, labels = dataiter.next()

    im = Image.fromarray((torch.cat(images.split(1,0),3).squeeze()/2*255+.5*255).permute(1,2,0).numpy().astype('uint8'))
    im.save("train_pt_images.jpg")
    print('train_pt_images.jpg saved.')
    print('Ground truth labels:' + ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))


    ## cnn
    net = Net()


    ## loss and optimiser
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


    ## train
    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Training done.')

    # save trained model
    torch.save(net.state_dict(), 'saved_model.pt')
    print('Model saved.')

In [ ]:
cgfhgvjhbkjnl

In [ ]:
inpt = np.random.rand(4, 3, 32, 32)
input_tensor = torch.Tensor(inpt)

conv1 = nn.Conv2d(3, 6, 5)
pool = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(6, 16, 5)
fc1 = nn.Linear(16 * 5 * 5, 120)
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 10)

print(input_tensor.shape)
x = pool(F.relu(conv1(input_tensor)))
print(x.shape)
x = pool(F.relu(conv2(x)))
print(x.shape)
print('drfytguhjn')
x = torch.flatten(x, 1) # flatten all dimensions except batch
print(x.shape)
x = F.relu(fc1(x))
print(x.shape)
x = F.relu(fc2(x))
print(x.shape)
x = fc3(x)
print(x.shape)

In [ ]:
===========
rnn = nn.LSTM(features_in=10, features_out=20, num_layers=1)
is similar to lstm = tf.keras.layers.LSTM(features_out=20)
h0 = torch.randn(2, 3, 20)
>>> c0 = torch.randn(2, 3, 20)
>>> output, (hn, cn) = rnn(input, (h0, c0))

===========

rnn = nn.LSTM(10, 20, 1, batch_first=True)

#  [batch, sequence length, features]
input = torch.randn(5, 3, 10)

h0 = torch.randn(1, 5, 20)
c0 = torch.randn(1, 5, 20)
output, (hn, cn) = rnn(input, (h0, c0))

# output shape: [batch, sequence length, out features]
output.shape
===============



In [ ]:
conv1 = nn.Conv2d(3, 6, 5) # Input channel size is 3 as the images have 3 colour channels, output channel size is 6 and the kernel size is 5
pool = nn.MaxPool2d(2, 2) # Kernel size of 2 and a stride of 2
conv2 = nn.Conv2d(6, 16, 5) # The input channel size is equal to the last output channel size
print(images.shape) # [ 4, 3, 32, 32] 4 batches, 3 colour channels (rgb) and images are 32x32
x = conv1(images)
print(x.shape) # [4, 6, 28, 28] 6 output channels as specified, image size is 28x28 (resulting image is smaller as calculated in line 88)
x = pool(x)
print(x.shape) # [4, 6, 14, 14] The pooling layer has a kernel size of 2x2 and a stride of 2. This reduces the images by a factor of 2
x = conv2(x)
print(x.shape) # [4, 16, 10, 10] 16 is the specified output size, 10 is calculated using the formula
x = pool(x)
print(x.shape) # [4, 16, 5, 5] The pooling layer has a kernel size of 2x2 and a stride of 2. This reduces the images by a factor of 2



In [ ]:
tensor_X_train.shape

In [ ]:
y_rating_arr = emg_data.rating_level_num.values
y_rating_arr = y_rating_arr[0::3]
y_rating_arr = np.reshape(y_rating_arr, (y_rating_arr.shape[0],1))
y_rating_arr.shape

In [ ]:
#my_x = [np.array([[1.0,2],[3,4]]),np.array([[5.,6],[7,8]])] # a list of numpy arrays
#my_y = [np.array([4.]), np.array([2.])] # another list of numpy arrays (targets)

tensor_X = torch.Tensor(my_x) # transform to torch tensor
tensor_y = torch.Tensor(my_y)

my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class NaturalSceneClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,6)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
dxtcfvgjbhknl

In [ ]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data/EMG_data_collection/emg_0_stats.csv"
emg_0 = pd.read_csv(path)
emg_0.head(5)

In [ ]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data/EMG_data_collection/emg_1_stats.csv"
emg_1 = pd.read_csv(path)
emg_1.head(5)

In [ ]:
int(emg_0_arr.shape[1])

In [ ]:
starting_index = 11
emg_0_arr = emg_0.iloc[:,starting_index:].values
print(emg_0_arr.shape)
reshaped_emg_0_arr = emg_0_arr.reshape((int(emg_0_arr.shape[0]/3), 3, int(emg_0_arr.shape[1])),order='F')
print(reshaped_emg_0_arr.shape)

emg_1_arr = emg_1.iloc[:,starting_index:].values
#print(emg_1_arr.shape)
reshaped_emg_1_arr = emg_1_arr.reshape((int(emg_1_arr.shape[0]/3), 3, int(emg_1_arr.shape[1])),order='F')
#print(reshaped_emg_1_arr.shape)


In [ ]:
emg_0_arr

In [ ]:
reshaped_emg_0_arr

In [ ]:
arr = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],[17,18,19,20],[21,22,23,24]])
print(arr)
print('-----------')
newarr = arr.reshape((2, 3, 4),order='A')
print(newarr)
print('-----------')
newarr = arr.reshape((2, 3, 4),order='C')
print(newarr)
print('-----------')
newarr = arr.reshape((2, 3, 4),order='F')
print(newarr)

In [ ]:
starting_index = 11
a = emg_0.iloc[:,starting_index:].values
a.shape

In [ ]:
np.random.rand(6,7,3)

In [ ]:
y_rating

In [ ]:
np.random.rand(3,2,4)

In [ ]:
starting_index = 11
X = emg_data.iloc[:,starting_index:].values
y = emg_data[['property_id', 'rating_level_num']].values
print(f'X.shape {X.shape}')
print(f'y.shape {y.shape}')
print('--------------------------------')

# Split the data into training and testing sets
X_train_property, X_test_property, y_train, y_test = train_test_split(X, y, test_size = 0.15, shuffle=True, random_state = num) 
print(f'X_train_property.shape {X_train_property.shape}')
print(f'X_test_property.shape {X_test_property.shape}')
print(f'y_train.shape {y_train.shape}')
print(f'y_test.shape {y_test.shape}')
print('--------------------------------')

y_train_property, y_train_ranking = np.hsplit(y_train,2)
y_test_property, y_test_ranking = np.hsplit(y_test,2)
print(f'y_train_property.shape {y_train_property.shape}')
print(f'y_test_property.shape {y_test_property.shape}')
print(f'y_train_ranking.shape {y_train_ranking.shape}')
print(f'y_test_ranking.shape {y_test_ranking.shape}')
print('--------------------------------')

X_train_ranking = np.concatenate((X_train_property, y_train_property), axis=1)
X_test_ranking = np.concatenate((X_test_property, y_test_property), axis=1)
print(f'X_train_ranking.shape {X_train_ranking.shape}')
print(f'X_test_ranking.shape {X_test_ranking.shape}')



In [ ]:
# Step 1: Build CNN+LSTM to predict the property

In [ ]:
a, b = np.hsplit(y_train,2)
b.shape

In [ ]:
y_property = np.reshape(y_property, (y_property.shape[0], 1))
y_rating = np.reshape(y_rating, (y_rating.shape[0], 1))
y_comb = np.concatenate((y_property, y_rating), axis=1)
X_train_property, X_test_property, y_train_property, y_test_property = train_test_split(X_property, y_comb, test_size = 0.15, shuffle=True, random_state = num) 



In [ ]:
print(X_property.shape)
print(y_property.shape)
b = 
print(b.shape)

a.shape

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot



In [ ]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '/Users/niharawarawita/Downloads/UCI HAR Dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + '/Users/niharawarawita/Downloads/UCI HAR Dataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(f'trainX.shape {trainX.shape}')
    print(f'trainy.shape {trainy.shape}')
    print(f'testX.shape {testX.shape}')
    print(f'testy.shape {testy.shape}')
   # print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

# load data
trainX, trainy, testX, testy = load_dataset()

In [ ]:
trainX.shape

In [ ]:
conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
>>> outputs = tf.keras.layers.TimeDistributed(conv_2d_layer)(inputs)
>>> outputs.shape
TensorShape([None, 10, 126, 126, 64])


In [ ]:
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')
output = TimeDistributed(conv1, input_shape=(None,n_length,n_features))(trainX)


In [ ]:
print(f'Before:trainX.shape {trainX.shape}')
n_timesteps, n_features= trainX.shape[1], trainX.shape[2]
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')
n_steps, n_length = 4, 32

print(f'trainX.shape before reshaping {trainX.shape}')
trainX_reshaped = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
print(f'trainX.shape after reshaping{trainX_reshaped.shape}')

output = TimeDistributed(conv1, input_shape=(None,n_length,n_features))(trainX_reshaped)
print(f'After conv1+temporal {output.shape}')

#output1 = TimeDistributed(conv1)(output)
#print(f'After conv1+temporal {output1.shape}')

   #model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
   # model.add(TimeDistributed(Dropout(0.5)))
   # model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
   # model.add(TimeDistributed(Flatten()))
   # model.add(LSTM(100))
   # model.add(Dropout(0.5))
   # model.add(Dense(100, activation='relu'))
   # model.add(Dense(n_outputs, activation='softmax'))



In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print(n_timesteps, n_features, n_outputs)
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    print(f'trainX.shape before {trainX.shape}')
    print(f'testX.shape before {testX.shape}')
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    print(f'trainX.shape after {trainX.shape}')
    print(f'testX.shape after {testX.shape}')
    
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

score = evaluate_model(trainX, trainy, testX, testy)
score

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy
 
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

evaluate_model(X_train, y_train, X_test, y_test)
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
 
# run the experiment
run_experiment()